# Mission to Mars

## Step 1 - Scraping

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time
import requests

### NASA Mars News

In [2]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
url = 'https://mars.nasa.gov/news/'
url_content = requests.get(url)
news_soup = BeautifulSoup(url_content.text)

# Assign the text to variables that you can reference later.
news_title = news_soup.find('div', class_='content_title').a.text.strip()
news_text = news_soup.find('div', class_="rollover_description_inner").text.strip()
mars_dict = {
            'title': news_title,
            'article': news_text
            }

### JPL Mars Space Images - Featured Image

In [3]:
# Visit the url for JPL Featured Space Image
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2)
browser.click_link_by_partial_text('more info')
img_html = browser.html
img_soup = BeautifulSoup(img_html, 'html.parser')

In [4]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url
featured_image_url = 'https:' + img_soup.find_all('div', class_="download_tiff")[1].p.a['href']

In [5]:
browser.quit()

### Mars Weather

In [6]:
# Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page
weather_url = 'https://twitter.com/marswxreport?lang=en'
weather_response = requests.get(weather_url)
weather_soup = BeautifulSoup(weather_response.text)

In [7]:
# Save the tweet text for the weather report as a variable called mars_weather
mars_weather = weather_soup.find('p', class_='tweet-text').contents[0]

### Mars Facts

In [8]:
# Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
facts_url = 'https://space-facts.com/mars/'

In [9]:
#Use Pandas to convert the data to a HTML table string.
table_df = pd.read_html(facts_url, index_col=0)[0]
html_table = table_df.to_html(header=False)
table_soup = BeautifulSoup(html_table)

### Mars Hemispheres

In [10]:
# Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
maps_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(maps_url)

In [11]:
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name.

hemisphere_links = browser.find_link_by_partial_text('Hemisphere')
hemisphere_image_urls = []

# Use a Python dictionary to store the data
for link in range(len(hemisphere_links)):
    browser.find_link_by_partial_text('Hemisphere')[link].click()
    img_url=browser.find_link_by_partial_text('Sample')['href']
    title = browser.find_by_tag('h2').text
# Append the dictionary with the image url string and the hemisphere title to a list.
    hemisphere_image_urls.append(
    {'title': title,
     'img_url': img_url
    })
    browser.back()
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [12]:
browser.quit()